In [1]:
import os
import torch
#os.environ["CUDA_VISIBLE_DEVICES"]=""
import sentencepiece as spm

from model import GeneratorSelfAttention
from torchUtils import get_model, get_tensor_data
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
n_embedding = 512
n_hidden = 512
n_layer = 8
batch_size = 32

sp = spm.SentencePieceProcessor()
sp.load('total.uni.35000.model')
model = GeneratorSelfAttention(sp.GetPieceSize(), n_embedding, n_hidden, n_layer)
print('build done')
model.load_state_dict(
    get_model('model/model.trans.{}.{}.th'.format(n_hidden, n_layer)))
model.cuda()
model.eval()
#model.share_memory()
print('model loaded')



build done
model loaded


In [54]:

def trim(prediction):
    if sp.GetPieceSize() + 1 in prediction:
        end = prediction.index(sp.GetPieceSize() + 1)
    elif 0 in prediction:
        end = prediction.index(0)
    else:
        end = len(prediction)
    prediction = prediction[0:end]
    return prediction


def translate(sequence):
    sequence = sequence.strip()
    ids = sp.EncodeAsIds(sequence)
    inputs = [None, torch.LongTensor([ids[0:100]]).cuda()] #.cuda()
    with torch.no_grad():
        prediction = model(inputs)
    output = get_tensor_data(prediction)
    prediction = trim(output[0].tolist())
    return sp.DecodeIds(prediction)

def translate_multi(seq_list):
    batch_ids = [sp.EncodeAsIds(seq.strip())[0:100] for seq in seq_list]
    #print(batch_ids)
    inps = torch.LongTensor(batch_ids).cuda()
    #print(input.shape)
    inputs = [None, inps]
    with torch.no_grad():
        prediction = model(inputs)
        
    batch_output = get_tensor_data(prediction)
    pred_texts = [sp.DecodeIds(trim(output.tolist())) for output in batch_output]
    return pred_texts

In [4]:
# ids = sp.EncodeAsIds('测试')
# inputs = [None, torch.LongTensor([ids[0:100]]).cuda()]
with torch.no_grad():
    prediction = model(inputs)

NameError: name 'inputs' is not defined

In [ ]:
torch.LongTensor([ids[0:100]]).shape

In [246]:
inputs = [None, torch.LongTensor([ids[0:100], ids[0:100]]).cuda()]

In [254]:
# print(prediction.shape) # (1, 70)
# output = get_tensor_data(prediction)
# print(output.shape) # (1,70)

trim(output[0].tolist())
trim(output[1].tolist())

[12765, 11]

In [5]:
import pandas as pd

# df_en = pd.read_csv('train_en.csv', encoding ='utf-8')
# df_tcn = pd.read_csv('train_tcn.csv', encoding ='utf-8')
df_test = pd.read_csv('test_tcn.csv', encoding ='utf-8')

df_para_en = pd.read_csv('dev_en.csv', encoding ='utf-8')
df_para_tcn = pd.read_csv('dev_tcn.csv', encoding ='utf-8')

In [6]:
import re
import regex
import opencc
import jieba
s2t_conv = opencc.OpenCC('s2t.json')

def s2t(s):
    return s2t_conv.convert(s)

t2s_conv = opencc.OpenCC('t2s.json')

def t2s(s):
    return t2s_conv.convert(s)

re.Pattern = regex.compile(r'\p{So}')


from nltk import word_tokenize

In [7]:
def preproc_tcn(text):
    clean = re.Pattern.sub(' ', text)
#     return s2t_conv.convert(clean)
    return clean

def preproc_en(text):
    return re.Pattern.sub(' ', text)

def preproc_tcn2(text):
    clean = re.Pattern.sub(' ', text)
    return t2s_conv.convert(clean)

In [10]:
print(df_para_tcn.columns, df_para_en.columns)

Index(['text', 'split'], dtype='object') Index(['translation_output'], dtype='object')


In [8]:
df_para_tcn['clean'] = df_para_tcn.apply(lambda x: preproc_tcn(x.text), axis=1)
# df_para_tcn['clean2'] = df_para_tcn.apply(lambda x: preproc_tcn2(x.text), axis=1)
df_para_en['clean'] = df_para_en.apply(lambda x: preproc_en(x.translation_output), axis=1)

In [65]:
df_test['clean'] = df_test.apply(lambda x: preproc_tcn(x.text), axis=1)

In [66]:
df_test.clean.isnull().any()

False

In [32]:
df_para_tcn[df_para_tcn.clean2 != df_para_tcn.text][['clean', 'clean2']]

,clean,clean2
1,SOFT 99 鍍膜車蠟(強力撥水型),SOFT 99 镀膜车蜡(强力拨水型)
2,低糖芒果乾 250g 臻御行,低糖芒果干 250g 臻御行
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,＊小径文化＊日本进口ROUND TOP space craft - diamond (SC-...
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,Hello Kitty 凯蒂猫 KITTY 凉鞋 童鞋 白/红色 小童 no739
5,雷鳥 LT-234 日式木紋桌墊 45 x 60 cm / 片,雷鸟 LT-234 日式木纹桌垫 45 x 60 cm / 片
...,...,...
994,力大牌 ABEL 66189 A4橫式板夾(顏色隨機出貨) / 個,力大牌 ABEL 66189 A4横式板夹(颜色随机出货) / 个
995,【HippoRed】撕破樂趣 獨特風格 中直筒牛仔褲 O591_445,【HippoRed】撕破乐趣 独特风格 中直筒牛仔裤 O591_445
996,兒童套裝 臺灣製薄長袖居家套裝 魔法Baby~k60092,儿童套装 台湾制薄长袖居家套装 魔法Baby~k60092
997,LONGCHAMP Le Pliage Neo高密尼龍後揹包(中型),LONGCHAMP Le Pliage Neo高密尼龙后背包(中型)


In [122]:
df_para_en.clean.iloc[997]

'LONGCHAMP Le Pliage Neo High Density Nylon Backpack (medium)'

In [123]:
word_tokenize(df_para_tcn.clean.iloc[997])

['LONGCHAMP', 'Le', 'Pliage', 'Neo高密尼龍後背包', '(', '中型', ')']

In [126]:
sent = df_para_tcn.clean.iloc[997]

sent = re.sub('】', '] ', sent)
sent = re.sub('【', ' [', sent)

for w in word_tokenize(sent):
    if re.search(u'[\u4e00-\u9fff]', w):
        #print(translate(w))
        #print(translate(t2s_conv.convert(w)))
        if re.search(u'[^\u4e00-\u9fff]', w):
#         w = translate(s2t_conv.convert(w))
            for xw in jieba.cut()
#         print(w)

    print(w)

LONGCHAMP
Le
Pliage
['Neo', '高密', '尼龍', '後', '背包']
Neo高密尼龍後背包
(
中型
)


In [160]:
df_para_en.clean.iloc[3]

'* the culture Japan Imported Round Top Space Craft - Diamond SC - MK - 010'

In [83]:
# global_cnt = 0

TRANS_MAP = {
    # size: {word: [(sent_index, word_index), ]}
}

def apply_translate(text_tcn, sent_index, ch_buffer_limit=2):
    t2s_conv = opencc.OpenCC('t2s.json')
    ch_string = '' # chinese string buffer
    en_words = []
    for w in jieba.cut(text_tcn):
        w = w.strip()
        if len(w) == 0:
            continue

        if re.search(u'[\u4e00-\u9fff]', w):
            ch_string += w
            if len(ch_string) >= ch_buffer_limit:
                #print(ch_string)
                #print(translate(t2s_conv.convert(ch_string)))
                #en_words.append(t2s_conv.convert(ch_string))
                
                word = t2s_conv.convert(ch_string)
                act_len = len(sp.EncodeAsIds(word.strip()))
                if act_len not in TRANS_MAP:
                    TRANS_MAP[act_len] = {}
                
                word = t2s_conv.convert(ch_string)
                if word not in TRANS_MAP[act_len]:
                    TRANS_MAP[act_len][word] = {'en': '', 'ixs':[]}
                
                TRANS_MAP[act_len][word]['ixs'].append((sent_index, len(en_words)))
                
                
                en_words.append('UNK')
                ch_string = ''

        else:
            if len(ch_string) > 0:
                #print(ch_string)
                #print(translate(t2s_conv.convert(ch_string)))
                #en_words.append(t2s_conv.convert(ch_string))
                
                word = t2s_conv.convert(ch_string)
                act_len = len(sp.EncodeAsIds(word.strip()))
                if act_len not in TRANS_MAP:
                    TRANS_MAP[act_len] = {}
                
                word = t2s_conv.convert(ch_string)
                if word not in TRANS_MAP[act_len]:
                    TRANS_MAP[act_len][word] = {'en': '', 'ixs':[]}
                
                TRANS_MAP[act_len][word]['ixs'].append((sent_index, len(en_words)))
                
                en_words.append('UNK')
                ch_string = ''
            else:
                en_words.append(w)

    if len(ch_string) > 0:
        #print(translate(t2s_conv.convert(ch_string)))
        #en_words.append(t2s_conv.convert(ch_string))
        
        word = t2s_conv.convert(ch_string)
        act_len = len(sp.EncodeAsIds(word.strip()))
        if act_len not in TRANS_MAP:
            TRANS_MAP[act_len] = {}

        word = t2s_conv.convert(ch_string)
        if word not in TRANS_MAP[act_len]:
            TRANS_MAP[act_len][word] = {'en': '', 'ixs':[]}

        TRANS_MAP[act_len][word]['ixs'].append((sent_index, len(en_words)))
        
        en_words.append('UNK')
        
#     global global_cnt
#     global_cnt += 1
#     print(global_cnt)
    return en_words

#print(apply_translate(df_para_tcn.clean.iloc[3]))

In [21]:
df_para_tcn.clean[0]

'OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳'

In [84]:
from tqdm import tqdm
to_rep = []
for i, sent in tqdm(enumerate(df_para_tcn.clean)):
    to_rep.append(apply_translate(sent, i))
    
#print(TRANS_MAP)

print(len(to_rep))


0it [00:00, ?it/s]
62it [00:00, 619.98it/s]
136it [00:00, 651.10it/s]
208it [00:00, 668.39it/s]
280it [00:00, 682.64it/s]
351it [00:00, 690.50it/s]
422it [00:00, 696.21it/s]
491it [00:00, 692.84it/s]
559it [00:00, 686.39it/s]
631it [00:00, 695.67it/s]
701it [00:01, 695.67it/s]
770it [00:01, 691.18it/s]
842it [00:01, 697.39it/s]
912it [00:01, 695.38it/s]
1000it [00:01, 695.76it/s][A

1000


In [85]:
for sz in TRANS_MAP:
    print(len(TRANS_MAP[sz].keys()))

1097
394
1454
49
46
4


In [86]:
BATCH_SIZE = 64

import math
for sz in TRANS_MAP:
    #print(sz)
    chs = list(TRANS_MAP[sz].keys())
    
    cnt = math.ceil(len(chs)/BATCH_SIZE)
    for i in tqdm(range(cnt)):
        st = i*BATCH_SIZE
        #print(chs[st:st+BATCH_SIZE])
        base = chs[st:st+BATCH_SIZE]
        res = translate_multi(base)
        for ch, en in zip(base, res):
            TRANS_MAP[sz][ch]['en'] = en


 17%|█▋        | 3/18 [00:54<04:31, 18.08s/it]

KeyboardInterrupt: 

In [74]:
# apply

for sz in TRANS_MAP:
    for ch in tqdm(TRANS_MAP[sz], position=0, leave=True):
        for s_ix, w_ix in TRANS_MAP[sz][ch]['ixs']:
            assert(to_rep[s_ix][w_ix] == 'UNK')
            to_rep[s_ix][w_ix] = TRANS_MAP[sz][ch]['en']
            


100%|██████████| 2/2 [00:00<00:00, 21620.12it/s]


In [77]:
df_final = pd.DataFrame(data=[' '.join(words) for words in to_rep], columns=['translation_output'])

In [82]:
df_final.translation_output

0       【 PolarStar 】 The beautiful No.  Warmer  ...a ...
1                     ~    The Pearl hand  The hand chain
2                Powder crystal  Purely lit Silver Mines 
3                  3M SCOTCH VHB Super powerful     V1808
4                                                    * 4 
                              ...                        
9995                RAINS BACKPACK Backback backpack     
9996             Airwalk Shadow shoes    A823230180 no002
9997                         NORNS 【  5000mAh   (   】    
9998                 DERWENT     12  6B - 4H ) ,  * 34214
9999                                    Borofone BE11    
Name: translation_output, Length: 10000, dtype: object

In [88]:
df_para_en.translation_output

0      Oppo A75 A75S A73 Phone Case Soft Rabbit Silic...
1              SOFT 99 Coating Car Wax Strong Water Watt
2                  Low Sugar Mango Dry 250g Be The Royal
3      * the culture Japan Imported Round Top Space C...
4      Hello Kitty Sandals Shoes White/Red Children n...
                             ...                        
995    Hippored Torn Fun Unique Style Straight Jeans ...
996    Kids Set Table Bay - Thin Long Sleeve Home Sui...
997    LONGCHAMP Le Pliage Neo High Density Nylon Bac...
998    IFairies Opening Adjustable Ring ifairies [564...
999    PolarStar Women Sweat Quick Dry T-shirt Black ...
Name: translation_output, Length: 1000, dtype: object

In [10]:
df_para_tcn['translation_output'] = df_para_tcn.apply(lambda x: apply_translate(x.text), axis=1)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
df_para_tcn.translation_output

0                  OPPO A75 A75s A73 手机 壳软壳 挂绳 壳大眼 兔硅胶 壳
1                                 SOFT 99 镀膜车 蜡 强力 拨水型 )
2                                        低糖 芒果干 250g 臻御行
3      ＊ 小径 文化 ＊ 日本 进口 ROUND TOP space craft - diamon...
4              Hello Kitty 凯蒂猫 KITTY 凉鞋 童鞋 白 红色 小童 no739
                             ...                        
995      【 HippoRed 】 撕破 乐趣 ★ 独特 风格 ★ 中直 筒牛仔裤 O591 _ 445
996                儿童 套装 台湾 制薄 长袖 居家 套装 魔法 Baby ~ k60092
997             LONGCHAMP Le Pliage Neo 高密 尼龙 后背包 ( 中型 )
998    iFairies 开口 可调节 戒指 ★ ifairies 【 56472 】 【 56472 】
999                       PolarStar 女排汗 快干 T恤 『 黑 P18102
Name: translation_output, Length: 1000, dtype: object

In [98]:
translate(t2s_conv.convert('白/紅色'))

'White/color'

In [74]:
df_para_en.clean.iloc[4]

'Hello Kitty Sandals Shoes White/Red Children no739'

In [53]:
translate('兒童套裝 臺灣製薄長袖居家套裝 魔法'), translate('儿童套装 台湾制薄长袖居家套装 魔法')

('The sylvanic trolley', 'Taiwan Mini-Spanel Saucer')

In [56]:
translate('兒童套裝'), translate('臺灣製薄長袖居家套裝')

("Children's clothing", 'The new')

In [62]:
list(jieba.cut('兒童套裝'))

['兒童', '套裝']

In [63]:
translate('兒童'), translate('套裝')

('Child of children', '...a condom of the new')

In [55]:
translate('儿童套装'), translate('台湾制薄长袖居家套装')

('Child Sets', 'Taiwan Mini-sized')

In [33]:


list(jieba.cut(df_para_tcn.text[0]))

['OPPO',
 ' ',
 'A75',
 ' ',
 'A75s',
 ' ',
 'A73',
 ' ',
 '手机',
 '壳',
 ' ',
 '软壳',
 ' ',
 '挂绳',
 '壳',
 ' ',
 '大眼',
 '兔',
 '硅胶',
 '壳']

In [43]:
for w in jieba.cut(df_para_tcn.text[5]):
    if w.strip() == '':
        continue
    
    print(translate(w))

thunderstorm
LT
-
234
Japanese style
Woody
Tables and
四十五、四十五
.
六十六十年
公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公公
//////////////////////////////////////////////////////////////////////
Tables of film


In [44]:
df_para_en.clean[5]

'Thunderbird Lt - 234 Japanese Wood Grain Table Mat 45 X 60 Cm / Pc'

In [168]:
import re
from typing import List

import regex
from sacrebleu import corpus_bleu

OTHERS_PATTERN: re.Pattern = regex.compile(r'\p{So}')


def eval(preds: List[str], refs: List[str]) -> float:
    """BLEU score computation.

    Strips all characters belonging to the unicode category "So".
    Tokenize with standard WMT "13a" tokenizer.
    Compute 4-BLEU.

    Args:
        preds (List[str]): List of translated texts.
        refs (List[str]): List of target reference texts.
    """
    preds = [OTHERS_PATTERN.sub(' ', text) for text in preds]
    refs = [OTHERS_PATTERN.sub(' ', text) for text in refs]
    return corpus_bleu(
        preds, [refs],
        lowercase=True,
        tokenize='13a',
        use_effective_order= False
    ).score

In [265]:
eval(df_para_tcn.translation_output.apply(lambda x: clean_output(x)), df_para_en.translation_output)

14.907776265225817

In [264]:
import string

def clean_output(text):
    text = text.replace('...a ', '').replace('...the ', '').replace('...', '')
    text = text.replace('????', '').replace('?', '')
    
    return text

#print(clean_output('Couples Short sleeves t Summer watery wash. 33 Number of figures Short sleeves t'))

In [90]:
samp = df_para_tcn.text.sample(n=5)
#print(samp.apply(lambda x: clean_output(x)).tolist())
print(samp)
print()
print(df_para_en.translation_output[samp.index].tolist())

#eval(samp, df_para_en.translation_output[samp.index])

923     厚底松糕底内增高旅游鞋原宿休闲女鞋 YX042813
344              民歌紀念日DVD 超值卡拉OK精選
745    禦寒保暖 名媛風一粒釦短大衣 艾爾莎【TAK6257】
765        Fujitsu F500-256GB 固態硬碟
984      紐頓 S6成犬 雞肉南瓜 2.72KG【寶羅寵品】
Name: text, dtype: object

['Thick Bottom Loose Bottom Travel Shoes Harajuku Shoes Yx042813', 'Folk Day Dvd Value Karaoke Picks', 'Winter Warm Ladies Wind A Button Topper Elsa 【 Tak6257 】', 'Fujitsu F500 - 256Gb SSD', 'Newton S6 Adult Dogs Chicken Pumpkin 2.72 Kg Paul Pet Products']


In [231]:
print(re.match('[a-zA-Z0-9]\w+(?:\.\w+)+', 'abc. '))

None


In [262]:
eval(['Roush - Slim Fit One single pocket pocket PETO shirt 715610'],
     ['Roush - Slim Fit Single Pocket Fit Plain Shirt 715610'])

31.7023313852343

In [190]:
print(samp[25])
print(samp[25].replace('...a ', '').replace('...the ', '').replace('...', ''))

Japan AZZEEN Stemie ...a planter. Queen's Queen ...a dressing water. 500ml ( 2 Editions 【 UR8D 】
Japan AZZEEN Stemie planter. Queen's Queen dressing water. 500ml ( 2 Editions 【 UR8D 】


In [261]:
df_para_tcn.to_csv('sample.csv', index=False)

In [277]:
len(df_test)

10000